# Research Recommendation Agent - 추론(Inference) 노트북

**2025 DATA·AI 분석 경진대회 - 논문·데이터 추천 에이전트**

이 노트북은 Qwen3-14B 기반 연구 데이터/논문 추천 시스템의 추론을 수행합니다.

## 실행 환경
- GPU: NVIDIA RTX 3080 이상 (INT8 양자화 시 14GB VRAM)
- CUDA: 11.8+
- Python: 3.10+

## 실행 순서
1. 환경 변수 설정
2. 모델 로드
3. 추론 실행
4. 결과 확인

## 1. 환경 설정 및 라이브러리 임포트

In [ ]:
import sys
import os

# 프로젝트 루트 경로 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_root)

print(f"프로젝트 루트: {project_root}")

# 필수 라이브러리 임포트
import asyncio
import json
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv(os.path.join(project_root, '.env'))
print("✅ 환경 변수 로드 완료")

## 2. GPU 및 CUDA 확인

In [ ]:
import torch

# GPU 확인
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA 버전: {torch.version.cuda}")
    print(f"사용 가능한 GPU 수: {torch.cuda.device_count()}")
    print(f"현재 GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️  GPU를 사용할 수 없습니다. CPU 모드 또는 DEV_MODE로 실행됩니다.")

## 3. 추천 에이전트 초기화

**중요**: 이 단계에서 Qwen3-14B 모델이 로드됩니다. GPU 메모리가 충분한지 확인하세요.
- INT4: ~8GB VRAM
- INT8: ~14GB VRAM
- FP16: ~28GB VRAM

In [ ]:
# 추천 에이전트 임포트 및 초기화
from src.agents.recommendation_agent import KoreanResearchRecommendationAgent
from src.config.settings import settings

print("모델 설정:")
print(f"  - 모델명: {settings.MODEL_NAME}")
print(f"  - 양자화: {settings.QUANTIZATION}")
print(f"  - 임베딩 모델: {settings.EMBEDDING_MODEL}")
print(f"  - 개발 모드: {settings.DEV_MODE}")
print("\n🚀 에이전트 초기화 중... (수 분 소요될 수 있습니다)")

agent = KoreanResearchRecommendationAgent()

print("\n✅ 에이전트 초기화 완료")
print(f"모델 정보: {json.dumps(agent.llm_model.get_model_info(), indent=2, ensure_ascii=False)}")

## 4. 테스트 데이터셋 ID 설정

DataON에 등록된 실제 데이터셋 ID를 입력하세요.

In [ ]:
# 테스트용 데이터셋 ID
# 예시: KISTI DataON의 실제 데이터셋 ID를 입력하세요
test_dataset_id = "SAMPLE_DATASET_ID"  # TODO: 실제 데이터셋 ID로 변경

print(f"테스트 데이터셋 ID: {test_dataset_id}")

## 5. 추론 실행

에이전트가 다음 단계를 수행합니다:
1. 소스 데이터셋 메타데이터 조회 (DataON API)
2. LLM으로 검색 쿼리 생성
3. 후보 수집 (DataON + ScienceON API)
4. 하이브리드 유사도 계산 (E5 + BM25)
5. LLM으로 최종 추천 생성

In [ ]:
# 추론 실행 (비동기)
import time

start_time = time.time()
print("🔍 추천 시작...\n")

# Jupyter에서 비동기 함수 실행
result = await agent.recommend(test_dataset_id)

elapsed_time = time.time() - start_time
print(f"\n✅ 추천 완료! (소요 시간: {elapsed_time:.2f}초)")

## 6. 결과 확인

In [ ]:
# 오류 확인
if 'error' in result:
    print(f"❌ 오류 발생: {result['error']}")
else:
    print("=" * 80)
    print("📊 추천 결과 요약")
    print("=" * 80)
    print(f"\n소스 데이터셋:")
    print(f"  ID: {result['source_dataset']['id']}")
    print(f"  제목: {result['source_dataset']['title']}")
    print(f"  키워드: {', '.join(result['source_dataset']['keywords'])}")
    
    print(f"\n추천 개수: {len(result['recommendations'])}개")
    print(f"분석 후보: {result['candidates_analyzed']}개")
    print(f"처리 시간: {result['processing_time_ms']}ms")
    
    print(f"\n모델 정보:")
    for key, value in result['model_info'].items():
        print(f"  {key}: {value}")
    
    print("\n" + "=" * 80)
    print("📝 추천 목록")
    print("=" * 80)

In [ ]:
# 추천 목록 상세 출력
if 'recommendations' in result:
    for rec in result['recommendations']:
        print(f"\n[{rec['rank']}위] {rec['level']} - {rec['type'].upper()}")
        print(f"제목: {rec['title']}")
        print(f"점수: {rec['score']:.3f}")
        print(f"이유: {rec['reason']}")
        print(f"URL: {rec['url']}")
        print("-" * 80)

## 7. JSON 파일로 결과 저장

In [ ]:
# 결과를 JSON 파일로 저장
output_dir = os.path.join(project_root, 'data', 'inference_results')
os.makedirs(output_dir, exist_ok=True)

output_file = os.path.join(output_dir, f"result_{test_dataset_id}.json")

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print(f"✅ 결과 저장 완료: {output_file}")

## 8. 리소스 정리

In [ ]:
# GPU 메모리 정리
if hasattr(agent, 'llm_model') and agent.llm_model:
    agent.llm_model.cleanup()
    print("✅ 모델 리소스 정리 완료")

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✅ GPU 메모리 캐시 정리 완료")

## 9. 추가 테스트 (선택사항)

여러 데이터셋에 대해 배치 추론을 수행할 수 있습니다.

In [ ]:
# 여러 데이터셋 ID 배치 테스트
test_dataset_ids = [
    "DATASET_ID_1",
    "DATASET_ID_2",
    "DATASET_ID_3"
]

batch_results = []

for dataset_id in test_dataset_ids:
    print(f"\n처리 중: {dataset_id}")
    try:
        result = await agent.recommend(dataset_id)
        batch_results.append({
            'dataset_id': dataset_id,
            'success': 'error' not in result,
            'result': result
        })
        print(f"✅ 완료: {len(result.get('recommendations', []))}개 추천")
    except Exception as e:
        print(f"❌ 실패: {e}")
        batch_results.append({
            'dataset_id': dataset_id,
            'success': False,
            'error': str(e)
        })

# 배치 결과 저장
batch_output_file = os.path.join(output_dir, 'batch_results.json')
with open(batch_output_file, 'w', encoding='utf-8') as f:
    json.dump(batch_results, f, ensure_ascii=False, indent=2)

print(f"\n✅ 배치 결과 저장 완료: {batch_output_file}")